# Initializing and Restoring EMI-RNN Graphs

The *EMI-RNN* implementation supports four forms of initialization/restoring:
1. An entirely new graph can be constructed with randomly initialized weights.
2. A saved graph can be loaded into the current `EMI_Driver`.
2. An entirely new graph can be constructed with weights initialized from a saved graph. This behavior is essentially restoration of a saved graph.
3. (*Experimental*) Initializing/Restoring using numpy matrices. 

All three methods will be illustrated in this notebook. This notebook uses the HAR dataset and builds on the [EMI LSTM example.ipynb](00_emi_lstm_example.ipynb)

In [1]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
# To include edgeml in python path
sys.path.insert(0, '../../')
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

# MI-RNN and EMI-RNN imports
from edgeml.graph.rnn import EMI_DataPipeline
from edgeml.graph.rnn import EMI_GRU
from edgeml.trainer.emirnnTrainer import EMI_Trainer, EMI_Driver
import edgeml.utils

Let us set up some network parameters for the computation graph.

In [2]:
# Network parameters for our LSTM + FC Layer
NUM_HIDDEN = 128
NUM_TIMESTEPS = 32
NUM_FEATS = 9
FORGET_BIAS = 1.0
NUM_OUTPUT = 3
USE_DROPOUT = True
KEEP_PROB = 0.75

# For dataset API
PREFETCH_NUM = 5
BATCH_SIZE = 32

# Number of epochs in *one iteration*
NUM_EPOCHS = 10
# Number of iterations in *one round*. After each iteration,
# the model is dumped to disk. At the end of the current
# round, the best model among all the dumped models in the
# current round is picked up..
NUM_ITER = 4
# A round consists of multiple training iterations and a belief
# update step using the best model from all of these iterations
NUM_ROUNDS = 2

# A staging direcory to store models
MODEL_PREFIX = '/home/iot/Documents/EdgeML-master/tf/examples/EMI-RNN/Daphnet328/model-gru'

# Loading Data

In [3]:
# Loading the data
path = "/home/iot/Documents/dataset_fog_release/dataset/32_8/"
x_train, y_train = np.load(path +'x_train.npy'), np.load(path +'y_train.npy')
x_test, y_test = np.load(path +'x_test.npy'), np.load(path +'y_test.npy')
x_val, y_val = np.load(path +'x_val.npy'), np.load(path +'y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_train.shape[1]
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

x_train shape is: (526, 4, 32, 9)
y_train shape is: (526, 4, 3)
x_test shape is: (59, 4, 32, 9)
y_test shape is: (59, 4, 3)


In [4]:
# Define the linear secondary classifier
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True
    
def addExtendedAssignOps(self, graph, W_val=None, B_val=None):
    W1 = graph.get_tensor_by_name('W1:0')
    B1 = graph.get_tensor_by_name('B1:0')
    W1_op = tf.assign(W1, W_val)
    B1_op = tf.assign(B1, B_val)
    self.assignOps.extend([W1_op, B1_op])

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
def feedDictFunc(self, keep_prob, **kwargs):
    feedDict = {self._emiGraph.keep_prob: keep_prob}
    return feedDict
    
EMI_GRU._createExtendedGraph = createExtendedGraph
EMI_GRU._restoreExtendedGraph = restoreExtendedGraph
EMI_GRU.addExtendedAssignOps = addExtendedAssignOps

def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index


if USE_DROPOUT is True:
    EMI_Driver.feedDictFunc = feedDictFunc

# ------ Skip from here ---------

## 1. Initializing a New Computation Graph

In the most common use cases, a new EMI-RNN graph would be created and trained

In [5]:
tf.reset_default_graph()

inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiLSTM = EMI_BasicLSTM(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                        forgetBias=FORGET_BIAS, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

# Construct the graph
g1 = tf.Graph()    
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)


NameError: name 'EMI_BasicLSTM' is not defined

Lets initialize a new session with this graph and train a model. The saved model will be used later for restoring.

In [ ]:
emiDriver.initializeSession(g1)
y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                      y_train=y_train, bag_train=BAG_TRAIN,
                                      x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                      numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                      numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                      numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                      fracEMI=0.5, updatePolicy='top-k', k=1)

As the output above indicates, the last restored model is `/tmp/model-lstm-1001`. That is, with `MODEL_PREFIX = '/tmp/model-lstm'` and `GLOBAL_STEP=1001`.

In [ ]:
def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index


k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))

## 2. Loading a Saved Graph into EMI-Driver

We will reset the computation graph, load a saved graph into the current `EMI_Driver` and verify its outputs.

In [ ]:
tf.reset_default_graph()

emiDriver.loadSavedGraphToNewSession(MODEL_PREFIX, 1001)
k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))


## 3. Initializing using a Saved Graph

Here we will construct a new computation graph, but will use a previously trained model to initialize it.

Use the `GraphManager` to load the saved graph and load it into a new session.

# ---resume here----

In [6]:
tf.reset_default_graph()
sess = tf.Session()
graphManager = edgeml.utils.GraphManager()
graph = graphManager.loadCheckpoint(sess, MODEL_PREFIX, globalStep=1004)

INFO:tensorflow:Restoring parameters from /home/iot/Documents/EdgeML-master/tf/examples/EMI-RNN/Daphnet328/model-gru-1004


Construct the forward graph as before, but provide the loaded `graph` as an argument to `__init__`.

In [7]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT, graph=graph)
emiLSTM = EMI_GRU(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                        useDropout=USE_DROPOUT, graph=graph)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy', graph=graph)

g1 = graph
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

Let `EMI_Driver` know that we already have a session in place.

In [8]:
##try changing k=1,2,3,4
emiDriver.setSession(sess)
k = 2

In [9]:
%%time


predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)


CPU times: user 1.05 s, sys: 142 ms, total: 1.2 s
Wall time: 695 ms


In [10]:
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))

Accuracy at k = 2: 0.880952


# -------END -------

## 4. Restoring from Numpy Matrices

We first extract the model matrices from the graph and dump it into `.npy` files. Then we load it back again and initialize a new graph with these matrices.

In [ ]:
graph = tf.get_default_graph()
W1 = graph.get_tensor_by_name('W1:0')
B1 = graph.get_tensor_by_name('B1:0')
allVars = emiLSTM.varList + [W1, B1]
sess = emiDriver.getCurrentSession()
allVars = sess.run(allVars)

base = '/tmp/models/'
np.save(base + 'kernel.npy', allVars[0])
np.save(base + 'bias.npy', allVars[1])
np.save(base + 'W1.npy', allVars[2])
np.save(base + 'B1.npy', allVars[3])

Reset the current session and graph

In [ ]:
sess = emiDriver.getCurrentSession()
sess.close()
tf.reset_default_graph()

Load the numpy matrices that will be used to initialize the graph.

In [ ]:
base = '/tmp/models/'
kernel = np.load(base + 'kernel.npy')
bias = np.load(base + 'bias.npy')
W = np.load(base + 'W1.npy')
B = np.load(base + 'B1.npy')

Proceed with graph construction as normally done, except that we add the requisite assignment operations.

In [ ]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS,
                                 NUM_OUTPUT)
emiLSTM = EMI_BasicLSTM(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                        forgetBias=FORGET_BIAS, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

tf.reset_default_graph()
graph = tf.Graph()

with graph.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    # Add the assignment operations
    emiLSTM.addBaseAssignOps(graph, [kernel, bias])
    emiLSTM.addExtendedAssignOps(graph, W, B)
    # Setup the driver. You can run the initializations manually as well
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

emiDriver.initializeSession(graph)
# Run the assignment operations
sess = emiDriver.getCurrentSession()
sess.run(emiLSTM.assignOps)

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test,
                                                               earlyPolicy_minProb,
                                                               minProb=0.99,
                                                               keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k,
                                             numClass=NUM_OUTPUT)
print('PART IV: Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions ==
                                                        BAG_TEST).astype(int))))